S-timator : demonstration of ODE models solving (uses the dynamics.py module).
------------------------------------------------------------------------------

This notebook shows how to use 4 of the most common S-timator functions:

- `read_model()`, reads a _string_ that conforms to a model description language, returning a `Model` object
- `solve()`, computes a solution of the ODE system associated with a model.
- `scan()`, calls Model.solve() several times, scanning a model parameter in a range of values.
- `plot()`, draws a graph of the results returned from `solve()` or `scan()`.


In [ ]:
%matplotlib inline

from stimator import read_model, examples
#from stimator.versions import version_info
#print ("Version information")
#print (version_info())
import matplotlib as mpl
import seaborn as snb

#To make sure we have always the same matplotlib settings
#(the ones in comments are the ipython notebook settings)

mpl.rcParams['figure.figsize']=(9.0,6.0)    #(6.0,4.0)
mpl.rcParams['font.size']=10                #10 
mpl.rcParams['savefig.dpi']=72             #72 

###Example 1
Glyoxalase system **model**


In [ ]:
mdl = examples.models.glyoxalases.text
print (mdl)
m1 = read_model(mdl)

s = m1.solve(tf=4030.0)
s.plot()

print ('==== Last time point ====')
print ('At t = %g'% s.t[-1])
for x in s.last:
    print ("%-8s= %f" % (x, s.last[x]))

###Example 2
Branched pathway

In [ ]:
from numpy import append, linspace
mdl = examples.models.branched.text; print mdl
m2 = read_model(mdl)

times = append(linspace(0.0, 5.0, 500), linspace(5.0, 10.0, 500))

m2.solve(tf=10.0, times=times).plot()

###Example 3
Calcium spikes: CICR model

In [ ]:
mdl = examples.models.ca.text; print (mdl)
#chaining functions...
read_model(mdl).solve(tf=8.0, npoints=2000).plot()

###Example 4
Rossler chaotic system

In [ ]:
mdl = examples.models.rossler.text; print (mdl)
m4 = read_model(mdl)

s = m4.solve(tf=100.0, npoints=2000, outputs="x1 x2 x3".split())

def transformation(vars, t):
    if t > 40.0:
        return (vars[0] - 5.0, vars[1], vars[2])
    else:
        return (-5.0, vars[1], vars[2])

s.apply_transf(transformation)

s.plot()

###Example 5
Lorentz chaotic system: sensitivity to initial conditions

In [ ]:
mdl = examples.models.lorentz.text
print (mdl)
m5 = read_model(mdl)

ivs = {'init.x':(1.0, 1.01, 1.02)}
titles = ['$x(0)$ = %g' % v for v in ivs['init.x']]
s = m5.scan(ivs, tf=25.0, npoints=20000, outputs=['x'], titles=titles)
s.plot(group='x', suptitlegend=m5.metadata['title'])

###Example 6
CICR model again: parameter scanning

In [ ]:
m = read_model("""
title Calcium Spikes
v0         = -> Ca, 1
v1         = -> Ca, k1*B*step(t, 1.0)
k1         = 7.3
B          = 0.4
export     = Ca ->  , 10 ..
leak       = CaComp -> Ca, 1 ..
!! Ca
v2         = Ca -> CaComp, 65 * Ca**2 / (1+Ca**2)
v3         = CaComp -> Ca, 500*CaComp**2/(CaComp**2+4) * Ca**4/(Ca**4 + 0.6561)
init       : (Ca = 0.1, CaComp = 0.63655)""")

mpl.rcParams['figure.subplot.hspace']=0.4   #.2

bvalues = (0.0, 0.1, 0.2, 0.25, 0.28, 0.29, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.75, 0.8, 0.9, 1.0)
titles = ['$\\beta$ = %g' % B for B in bvalues]

s = m.scan({'B': bvalues}, tf=8.0, npoints=1000)
suptitlegend="Dynamics of cytosolic $Ca^{2+}$ as a function of stimulus"
s.plot(yrange=(0,1.5), legend=False, fig_size=(13.0,13.0), titles=titles, suptitlegend=suptitlegend)